Плющевский Андрей, мФТиАД-2019

In [1]:
import numpy as np
import multiprocessing
from itertools import product
from IPython import display

import warnings
warnings.filterwarnings('ignore')

### Задание 1. Вычисление числа пи методом Монте-Карло

In [2]:
%%time
#Реализация через цикл
r=0
n = 10000000
for _ in range(n):
    x1 = np.random.random()
    x2 = np.random.random()
    if x1**2 + x2**2 <= 1:
        r+=1
r = r*4 / n
print(r)

3.1428324
Wall time: 26 s


In [3]:
%%time
#Векторная реализация
n = 10000000
x1 = np.random.uniform(0, 1, n)
x2 = np.random.uniform(0, 1, n)
r = np.sum(np.square(x1) + np.square(x2) <= 1) * 4 / n
print(r)

3.1420144
Wall time: 916 ms


Векторная реализация оказалась в ~20 раз быстрее цикла при значении n = 10000000

### Задание 3. Перемножение матриц

In [10]:
A = np.random.randint(1000, size=(500, 100))
B = np.random.randint(1000, size=(100, 50))
pool = multiprocessing.Pool(8)

In [11]:
#Функции в папке util.py:
'''
def prod(x1, x2):
    return np.sum((i*j) for (i,j) in zip(x1,x2))

def row_prod(row, matrix):
    r = []
    for i in range(matrix.shape[1]):
        col = [matrix[k][i] for k in range(len(matrix))]
        r.append(prod(row, col))
    return r
'''
from util import *

In [12]:
def matrix_prod(A, B, multiproc=True):
    if multiproc:
        return pool.starmap(row_prod, product(A, [B]))
    else:
        res = []
        for i in range(len(A)):
            res.append(row_prod(A[i], B))
        return np.array(res)

In [15]:
%%time
assert(matrix_prod(A, B, False)==A@B).all()

Wall time: 4.43 s


In [16]:
%%time
if __name__ ==  '__main__':
    assert(matrix_prod(A, B, True)==A@B).all()

Wall time: 2.01 s


При матрицах размером 500x100 и 100x50 (больше я на своём калькуляторе боюсь делать, а то сгорит) параллельное перемножение оказалось в ~2 раза быстрее последовательного 

### Задание 4. Сортировка

In [47]:
def sort_1(lst):
    if len(lst) < 2:
        return lst
    res = []
    mid = len(lst) // 2
    l = sort_1(lst[:mid])
    r = sort_1(lst[mid:])
    while (len(l) > 0) and (len(r) > 0):
        if l[0] > r[0]:
            res.append(r[0])
            r.pop(0)
        else:
            res.append(l[0])
            l.pop(0)
    res += l
    res += r
    return res

In [52]:
%%time
lst = [i for i in range(10)]
np.random.shuffle(lst)

print(sort_1(lst))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Wall time: 996 µs


In [54]:
'''
def sort_2(lst, end=None):
    if len(lst) < 2:
        res = lst
    else:
        mid = len(lst) // 2

        inputs = [lst[:mid], lst[mid:]]
        pipes = [multiprocessing.Pipe(False) for _ in inputs]
        processes = [multiprocessing.Process(target=sort_2, args=(input, end))
                    for input, (get_end, end) in zip(inputs, pipes)]
        for process in processes: process.start()
        for process in processes: process.join()
        result = [get_end.recv() for get_end, end in pipes]
        
        res = list(merge(*result))

    if end:
        end.send(res)
    else:
        return res
'''
from util import sort_2

In [55]:
%%time
lst = [i for i in range(10)]
np.random.shuffle(lst)

if __name__ ==  '__main__':
    print(sort_2(lst))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Wall time: 11.2 s
